# PatentSBERTa Model Test On AI-Growth-Labs Test Data

## Original Repo: https://github.com/AI-Growth-Lab/PatentSBERTa/blob/main/PatentSBERTa_Example_For_Patent_Classification.ipynb

In [1]:
!pip install datasets -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import datasets

import pandas as pd

In [3]:
train_dataset = datasets.load_dataset("AI-Growth-Lab/patents_claims_1.5m_traim_test", split="train[:100]")
test_dataset = datasets.load_dataset("AI-Growth-Lab/patents_claims_1.5m_traim_test", split="test[:10]")

df_claim_cpc_test = pd.DataFrame(test_dataset)
df_claim_cpc_train = pd.DataFrame(train_dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
df_claim_cpc_test.head()


,id,date,text,A01B,A01C,A01D,A01F,A01G,A01H,A01J,...,Y02B,Y02C,Y02D,Y02E,Y02P,Y02T,Y02W,Y04S,Y10S,Y10T
0,9289350,2016-03-22,1. An apparatus for generating air pressure an...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9579560,2017-02-28,1. A user-customizable audio-visual puzzle com...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9269063,2016-02-23,1. A system comprising: circuitry configured t...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9576958,2017-02-21,1. A semiconductor structure comprising: a sem...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8441548,2013-05-14,"1. A method comprising: capturing, by a camera...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_claim_cpc_test.shape


(10, 666)

In [6]:
df_claim_cpc_train.shape


(100, 666)

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')

#Sentences are encoded by calling model.encode()
test_embeddings = model.encode(df_claim_cpc_test.text, convert_to_tensor=True, show_progress_bar=True)
claim_embeddings = model.encode(df_claim_cpc_train.text, convert_to_tensor=True, show_progress_bar=True)
stored_patent_test_embeddings_id = df_claim_cpc_test['id']
stored_patent_train_embeddings_id = df_claim_cpc_train['id']

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# get_ipython().system('pip install sentence_transformers')
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tokenizers import Tokenizer
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)
import pandas as pd
import torch
import random
import itertools
import pickle
import time

start = time.time()

F1Measure_list = []
Recall_list = []
Accuracy_list = []
Precision_list = []
Hamming_Loss_list = []


def get_top_n_similar_patents_df(new_claim, claim_embeddings):
    search_hits_list = []
    search_hits = util.semantic_search(new_claim, claim_embeddings, 10000, 5000000, 20)
    top_claim_order = []
    top_claim_ids = []
    top_similarity_scores = []
    for item in range(len(search_hits[0])):
        top_claim_order = search_hits[0][item].get('corpus_id')
        top_claim_ids.append(stored_patent_train_embeddings_id[top_claim_order])
        top_similarity_scores.append(search_hits[0][item].get('score'))

    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'cosine_similarity': top_similarity_scores,
    })



    return top_100_similar_patents_df

def F1Measure(y_true, y_pred):
    save_F1 = []
    temp = 0
    for i in range(y_true.shape[0]):
        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
            continue
        temp_save = (2*sum(np.logical_and(y_true[i], y_pred[i])))/ (sum(y_true[i])+sum(y_pred[i]))
        save_F1.append(temp_save)
        temp += temp_save

    save_F1 = pd.DataFrame(save_F1)
    save_F1_ids = pd.concat([result, save_F1], axis=1, ignore_index=True)
    return temp/ y_true.shape[0]

def Recall(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_pred[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_pred[i])
    return temp/ y_true.shape[0]

def Precision(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_true[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_true[i])
    return temp/ y_true.shape[0]

def Hamming_Loss(y_true, y_pred):
    temp=0
    for i in range(y_true.shape[0]):
        temp += np.size(y_true[i] == y_pred[i]) - np.count_nonzero(y_true[i] == y_pred[i])
    return temp/(y_true.shape[0] * y_true.shape[1])

def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0]



claims = list(df_claim_cpc_train.text)
patent_id = list(df_claim_cpc_train.id)

listofpredictdfs = []

start = time.time()

for i in range(len(df_claim_cpc_test[:100])):
    get_top_n_similar_patents_df_predict = get_top_n_similar_patents_df(np.array(test_embeddings[i]).reshape(1,-1), claim_embeddings)
    result = pd.merge(get_top_n_similar_patents_df_predict, df_claim_cpc_train, left_on='top_claim_ids',right_on='id',how='left',suffixes=('_left','_right'))
    locals()["predict_n"+str(i)] = result.copy()
    listofpredictdfs.append("predict_n"+str(i))

df = pd.concat(map(lambda x: eval(x), listofpredictdfs),keys= listofpredictdfs ,axis=0)

top_k = 20
for k in range(top_k):
    top_n = k
    predict = pd.DataFrame(columns= df_claim_cpc_test.columns[6:])
    for item in range(len(listofpredictdfs)):
        k_similar_patents = df.xs(listofpredictdfs[item]).nlargest(top_n, ['cosine_similarity'])
        result_k_similar_patents = pd.DataFrame(0, index=np.arange(1),columns= k_similar_patents.columns[8:])
        for i in range(top_n):
            result_k_similar_patents  = result_k_similar_patents + k_similar_patents.iloc[i, 8:].values
        result_k_similar_patents_df = pd.DataFrame(result_k_similar_patents, columns= k_similar_patents.columns[8:])
        result_k_similar_patents_df.insert(0, "input_aptent_id", df_claim_cpc_test.id.iloc[item], True)
        locals()["predict"+str(item)] = result_k_similar_patents_df.copy()
        predict = pd.concat([predict, locals()["predict"+str(item)]], ignore_index=True)
        result_k_similar_patents_df = result_k_similar_patents_df[0:0]
    data = torch.tensor((predict.to_numpy()).astype(float), dtype=torch.float32)
    m = nn.Sigmoid()
    output = m(data)
    output = (output>0.9).float()
    output_df = pd.DataFrame(output, columns=predict.columns).astype(float)
    y_pred = output_df.iloc[:, :-1].to_numpy()
    y_true = df_claim_cpc_test.iloc[:100, 6:].to_numpy()
    result = pd.concat([output_df, df_claim_cpc_test], axis=1, ignore_index=True)
    F1Measure_list.append(F1Measure(y_true,y_pred))
    Recall_list.append(Recall(y_true,y_pred))
    Accuracy_list.append(Accuracy(y_true, y_pred))
    Precision_list.append(Precision(y_true,y_pred))
    Hamming_Loss_list.append(Hamming_Loss(y_true, y_pred))
    end = time.time()
    print(f"Runtime of the program is {end - start}")
    print("F1Measure: ", F1Measure_list[top_n])
    print("Recall: ", Recall_list[top_n])
    print("Accuracy: ", Accuracy_list[top_n])
    print("Precision: ", Precision_list[top_n])
    print("Hamming_Loss: ", Hamming_Loss_list[top_n])

output_d_metrics = {'F1Measure':F1Measure_list,'Recall_list':Recall_list, 'Accuracy_list':Accuracy_list,'Precision_list':Precision_list,'Hamming_Loss_list':Hamming_Loss_list}
output_df_metrics = pd.DataFrame(output_d_metrics)

Runtime of the program is 3.0522396564483643
F1Measure:  0.0
Recall:  0.0
Accuracy:  0.0
Precision:  0.0
Hamming_Loss:  0.0022727272727272726
Runtime of the program is 3.48357892036438
F1Measure:  0.0
Recall:  0.0
Accuracy:  0.0
Precision:  0.0
Hamming_Loss:  0.0022727272727272726
Runtime of the program is 4.060601711273193
F1Measure:  0.0
Recall:  0.0
Accuracy:  0.0
Precision:  0.0
Hamming_Loss:  0.0022727272727272726
Runtime of the program is 4.722742319107056
F1Measure:  0.1
Recall:  0.1
Accuracy:  0.1
Precision:  0.1
Hamming_Loss:  0.0021212121212121214
Runtime of the program is 5.344782590866089
F1Measure:  0.1
Recall:  0.1
Accuracy:  0.1
Precision:  0.1
Hamming_Loss:  0.0022727272727272726
Runtime of the program is 5.72593355178833
F1Measure:  0.1
Recall:  0.1
Accuracy:  0.1
Precision:  0.1
Hamming_Loss:  0.0024242424242424242
Runtime of the program is 5.947091579437256
F1Measure:  0.26666666666666666
Recall:  0.26666666666666666
Accuracy:  0.25
Precision:  0.26666666666666666
Ha